# Code is adapted from Kaggle

In [ ]:
#!pip install numpy==1.22.4

In [1]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

from dotenv import load_dotenv
import os
from openai import OpenAI
import time
import glob

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.model_selection import train_test_split
import pathlib
from sklearn.utils import shuffle


import warnings
warnings.filterwarnings('ignore')

load_dotenv()

my_id = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key=os.environ.get("my_id"),
)

In [2]:
dataframe = pd.read_csv("/Users/roulierk/Documents/Clemson/Thesis/Seven-Phishing-email-data/Enron.csv")
dataframe = shuffle(dataframe)
dataframe.loc[dataframe["label"]==0, "label"] = 0
dataframe.loc[dataframe["label"]==1, "label"] = 1
dataframe["label"] = dataframe["label"].astype(float)

In [4]:
#print(dataframe['label'].to_string(index=False))

In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29767 entries, 2830 to 19632
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   subject  29569 non-null  object 
 1   body     29767 non-null  object 
 2   label    29767 non-null  float64
dtypes: float64(1), object(2)
memory usage: 930.2+ KB


### Reduce the dataset to 10.  Don't need all of 11k elements for this.

In [6]:
# Getting first 10 rows from dataframe 
df_10 = dataframe.head(10) 
  
#  Printing df_10 
df_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 2830 to 17057
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   subject  10 non-null     object 
 1   body     10 non-null     object 
 2   label    10 non-null     float64
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


### Using the reduce dataset (10), Clean up the data, removing zeros and ones, remove "status"

In [7]:
allColumns = list(df_10.columns)
print(allColumns)

['subject', 'body', 'label']


### Add defaults to the file so that later we can play with temp, etc.

In [8]:
new_df = df_10.assign(Legit=np.NaN, Phish=np.NaN, Unknown=np.NaN,temp=0, elapsed=0)  #add the defaults for now

In [9]:
new_df

,subject,body,label,Legit,Phish,Unknown,temp,elapsed
2830,re : midcon transport usage tickets - - marol,bueno . . . it ' s done .\ndaren j farmer @ ec...,0.0,NaN,NaN,NaN,0,0
22780,online approval for $ 5000 now,"sincerely , your new offers staff\nremove\n",1.0,NaN,NaN,NaN,0,0
18091,femmerol - clinically proven to fight pms,to unsubscribe paliourg @ iit . demokritos . g...,1.0,NaN,NaN,NaN,0,0
104,"hpl nom for november 10 , 2000",( see attached file : hplnl 110 . xls )\n- hpl...,0.0,NaN,NaN,NaN,0,0
15567,here you go,enough door eat know mount . differ old here o...,1.0,NaN,NaN,NaN,0,0
21701,re : ebs position,"june 16 full - time , but some half days befor...",0.0,NaN,NaN,NaN,0,0
23263,from bar . stephen kwesi,barrister stephen kwesi\nkwesi consultants\nac...,1.0,NaN,NaN,NaN,0,0
13215,ena trading track,"all ,\nthank you for your time yesterday inter...",0.0,NaN,NaN,NaN,0,0
21228,re : global operations controller forum,completed as below\nenron capital & trade reso...,0.0,NaN,NaN,NaN,0,0
17057,start date : 1 / 15 / 02 ; hourahead hour : 6 ;,start date : 1 / 15 / 02 ; hourahead hour : 6 ...,0.0,NaN,NaN,NaN,0,0


### Create a new "prompt" file to drive the ChatGPT.

In [10]:
new_df.to_csv('/Users/roulierk/Documents/Clemson/Thesis/Seven-Phishing-email-data/test_email_prompt1.csv', header=False, index=False, index_label=None, mode='w')  